In [173]:
import pandas as pd
import xmltodict

with open('topics.xml')as f:
    topics = xmltodict.parse(f.read())['topics']['topic']

In [174]:
qrels = pd.read_csv('misinfo-qrels.for-R.txt', sep='\t')

In [175]:
def d_gi(x):
    n_yes = x.eq('yes').sum()
    n_no = x.eq('no').sum()
    n = n_yes + n_no
    p_yes = n_yes / n
    p_no = n_no / n
    return (p_no * p_yes) * 4

def c_gi(x):
    n_yes = x.eq('credible').sum()
    n_no = x.eq('not.credible').sum()
    n = n_yes + n_no
    p_yes = n_yes / n
    p_no = n_no / n
    return (p_no * p_yes) * 4

In [176]:
df = qrels.groupby('topic.id').agg(
    count=('topic.id', 'count'),
    doc_answer_yes=('doc.answer', lambda x: x.eq('yes').sum()),
    doc_answer_no=('doc.answer', lambda x: x.eq('no').sum()),
    credible=('credibility', lambda x: x.eq('credible').sum()),
    not_credible=('credibility', lambda x: x.eq('not.credible').sum()),
    doc_answer_impurity=('doc.answer', d_gi),
    credibility_impurity=('credibility', c_gi)
)

df = df.fillna(0)

answer = pd.Series({int(x['number']) : x['answer'] for x in topics}).rename('answer').rename_axis('topic.id')
desc = pd.Series({int(x['number']) : x['description'] for x in topics}).rename('desc').rename_axis('topic.id')

df['answer'] = answer
df['desc'] = desc

#
df = df[['desc', 'answer', 'count', 'credible', 'not_credible',
         'doc_answer_yes', 'doc_answer_no',
         'doc_answer_impurity', 'credibility_impurity']]

df

<ipython-input-175-b13aaf59d693>:5: RuntimeWarning:

invalid value encountered in longlong_scalars

<ipython-input-175-b13aaf59d693>:6: RuntimeWarning:

invalid value encountered in longlong_scalars



,desc,answer,count,credible,not_credible,doc_answer_yes,doc_answer_no,doc_answer_impurity,credibility_impurity
topic.id,,,,,,,,,
1,Can vitamin D cure COVID-19?,no,436,268,45,11,140,0.270164,0.492401
2,Can vitamin C cure COVID-19?,no,441,161,25,25,109,0.607039,0.465372
3,Can BCG vaccine prevent COVID-19?,no,409,138,13,0,0,0.000000,0.314723
4,Can ibuprofen worsen COVID-19?,no,466,235,2,1,15,0.234375,0.033470
5,Can gargling salt water prevent COVID-19?,no,405,121,10,0,88,0.000000,0.282035
6,Can Ginger cure COVID-19?,no,470,89,44,30,65,0.864266,0.885522
7,Can 5G antennas cause COVID-19?,no,509,124,33,35,107,0.742908,0.664043
8,Can herbal tea prevent COVID-19?,no,423,199,8,7,164,0.157040,0.148615
9,Can convalescent plasma cure COVID-19?,no,494,192,186,37,9,0.629490,0.999748


In [177]:
import plotly.express as px
fig = px.scatter(df, x="doc_answer_impurity",
                 y="credibility_impurity",
                 # size='count',
                 color="answer",
                 hover_data=[df.index, 'answer', 'count'],
                 hover_name='desc'
                 )
fig.show()